In [1]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
26,application_1607949680860_0028,pyspark,idle,Link,Link


SparkSession available as 'spark'.

In [2]:
import hashlib
from datetime import datetime
from pyspark.sql import functions as func
from pyspark.sql.types import FloatType
import hsfs

In [3]:
# Create a connection
connection = hsfs.connection()
# Get the feature store handle for the project's feature store
fs = connection.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

## Placeholder

In [4]:
transactions_fg = fs.get_feature_group("transactions_fg", 1)

In [5]:
transactions_df = transactions_fg.read()
transactions_df.show()

+------+-------+------+--------+------+--------------+-------+--------+
|is_sar|tx_type|source|alert_id|target|tran_timestamp|tran_id|base_amt|
+------+-------+------+--------+------+--------------+-------+--------+
|     0|      4|  1767|      -1|  3259|   1.4832288E9|      1| 9405.71|
|     0|      4|  7363|      -1|  5141|   1.4832288E9|      2| 6884.54|
|     0|      4|  7585|      -1|  9532|   1.4832288E9|      3|  7968.4|
|     0|      4|  1750|      -1|  8792|   1.4832288E9|      4| 9042.67|
|     0|      4|  9060|      -1|  4670|   1.4832288E9|      5| 4692.79|
|     0|      4|  8752|      -1|  3861|   1.4832288E9|      6| 4089.65|
|     0|      4|  9645|      -1|  3805|   1.4832288E9|      7| 3055.04|
|     0|      4|  7150|      -1|  9280|   1.4832288E9|      8| 6473.45|
|     0|      4|  1685|      -1|  6369|   1.4832288E9|      9| 2583.42|
|     0|      4|  7846|      -1|  8255|   1.4832288E9|     10| 6753.04|
|     0|      4|   878|      -1|  5957|   1.4832288E9|     11| 1

## Placeholder

In [6]:
edges_td_meta = fs.create_training_dataset(name="edges_td",
                                       version=1,
                                       data_format="csv",
                                       label = ["is_sar"],   
                                       description="edges training dataset")
edges_td_meta.save(transactions_df)

### Placeholder

In [7]:
account_fg = fs.get_feature_group("account_features", 1)

In [8]:
accounts_df = account_fg.read()
accounts_df.show()

+---------------+--------------+---------+-------+
|initial_deposit|tx_behavior_id|prior_sar|acct_id|
+---------------+--------------+---------+-------+
|       84442.19|             1|        0|      0|
|       75795.44|             1|        0|      1|
|       42057.16|             1|        0|      2|
|       25891.68|             1|        0|      3|
|       51127.47|             1|        0|      4|
|       40493.41|             1|        0|      5|
|       78379.86|             1|        0|      6|
|       30331.27|             1|        0|      7|
|        47659.7|             1|        0|      8|
|        58338.2|             1|        0|      9|
|       90811.29|             1|        0|     10|
|       50468.69|             1|        0|     11|
|       28183.78|             1|        0|     12|
|       75580.42|             1|        0|     13|
|        61836.9|             1|        0|     14|
|       25050.63|             1|        0|     15|
|       90974.63|             1

### Placeholder

In [9]:
source = transactions_fg.select(["source","is_sar"]).read().withColumnRenamed("source","id")
target = transactions_fg.select(["target","is_sar"]).read().withColumnRenamed("target","id")
nodes = source.union(target)
sar_nodes = nodes.where(nodes.is_sar == 1)
non_sar = nodes.where(nodes.is_sar == 0)
non_sar_nodes = non_sar.join(sar_nodes, ["id"], "leftanti")
labels = sar_nodes.union(non_sar_nodes).dropDuplicates(subset=["id"])
labels.show()

+----+------+
|  id|is_sar|
+----+------+
|3997|     1|
|5300|     1|
| 463|     1|
|8086|     1|
|7833|     1|
|1088|     0|
|6654|     0|
|9465|     0|
|7880|     0|
|6336|     0|
|1580|     0|
|1829|     0|
|7993|     0|
|7240|     0|
|7253|     0|
|8389|     0|
|4935|     0|
|6658|     0|
|6466|     0|
|8592|     0|
+----+------+
only showing top 20 rows

In [10]:
accounts_df = accounts_df.withColumnRenamed("acct_id","id")
accounts_df.show()

+---------------+--------------+---------+---+
|initial_deposit|tx_behavior_id|prior_sar| id|
+---------------+--------------+---------+---+
|       84442.19|             1|        0|  0|
|       75795.44|             1|        0|  1|
|       42057.16|             1|        0|  2|
|       25891.68|             1|        0|  3|
|       51127.47|             1|        0|  4|
|       40493.41|             1|        0|  5|
|       78379.86|             1|        0|  6|
|       30331.27|             1|        0|  7|
|        47659.7|             1|        0|  8|
|        58338.2|             1|        0|  9|
|       90811.29|             1|        0| 10|
|       50468.69|             1|        0| 11|
|       28183.78|             1|        0| 12|
|       75580.42|             1|        0| 13|
|        61836.9|             1|        0| 14|
|       25050.63|             1|        0| 15|
|       90974.63|             1|        0| 16|
|       98278.55|             1|        0| 17|
|       81021

In [11]:
accounts_df.count()

10000

In [12]:
nodes_df = accounts_df.join(labels, ["id"])
nodes_df.count()

9998

In [13]:
node_td = fs.create_training_dataset(name="node_td",
                                       version=1,
                                       data_format="csv",   
                                       description="node training dataset")
node_td.save(nodes_df)